<a href="https://colab.research.google.com/github/blhprasanna99/speech_emotion_detection/blob/master/CNN_SpeechEmotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install soundfile

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import soundfile
import numpy as np
import librosa
import glob
import os
from sklearn.model_selection import train_test_split

# all emotions on RAVDESS dataset
int2emotion = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

# we allow only these emotions
AVAILABLE_EMOTIONS = {
    "angry",
    "sad",
    "neutral",
    "happy"
}


In [0]:
def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result

In [0]:
def load_data(test_size=0.2):
    X, y = [], []
    try :
      for file in glob.glob("/content/drive/My Drive/wav/Actor_*/*.wav"):
          # get the base name of the audio file
          basename = os.path.basename(file)
          print(basename)
          # get the emotion label
          emotion = int2emotion[basename.split("-")[2]]
          # we allow only AVAILABLE_EMOTIONS we set
          if emotion not in AVAILABLE_EMOTIONS:
              continue
          # extract speech features
          features = extract_feature(file, mfcc=True, chroma=True, mel=True)
          # add to data
          X.append(features)
          l={'happy':0.0,'sad':1.0,'neutral':3.0,'angry':4.0}
          y.append(l[emotion])
    except :
         pass
    # split the data to training and testing and return it
    return train_test_split(np.array(X), y, test_size=test_size, random_state=7)

In [0]:
X_train, X_test, y_train, y_test = load_data(test_size=0.25)

print("[+] Number of training samples:", X_train.shape[0])
# number of samples in testing data
print("[+] Number of testing samples:", X_test.shape[0])

03-01-01-01-01-01-21.wav
03-01-01-01-02-02-21.wav
03-01-01-01-01-02-21.wav
03-01-02-01-01-01-21.wav
03-01-02-01-01-02-21.wav
03-01-02-01-02-01-21.wav
03-01-01-01-02-01-21.wav
03-01-02-01-02-02-21.wav
03-01-02-02-01-01-21.wav
03-01-02-02-02-01-21.wav
03-01-02-02-02-02-21.wav
03-01-02-02-01-02-21.wav
03-01-03-01-02-02-21.wav
03-01-03-01-01-01-21.wav
03-01-03-01-02-01-21.wav
03-01-03-01-01-02-21.wav
03-01-03-02-02-02-21.wav
03-01-03-02-01-01-21.wav
03-01-03-02-02-01-21.wav
03-01-03-02-01-02-21.wav
03-01-04-01-01-01-21.wav
03-01-04-01-01-02-21.wav
03-01-04-01-02-01-21.wav
03-01-04-01-02-02-21.wav
03-01-05-01-01-02-21.wav
03-01-04-02-01-01-21.wav
03-01-04-02-01-02-21.wav
03-01-05-01-02-01-21.wav
03-01-04-02-02-02-21.wav
03-01-05-01-01-01-21.wav
03-01-05-02-01-01-21.wav
03-01-05-01-02-02-21.wav
03-01-04-02-02-01-21.wav
03-01-06-01-02-02-21.wav
03-01-05-02-02-01-21.wav
03-01-06-02-01-01-21.wav
03-01-05-02-01-02-21.wav
03-01-06-02-01-02-21.wav
03-01-06-02-02-01-21.wav
03-01-06-01-01-01-21.wav


In [0]:
import numpy as np
X_train = np.asarray(X_train)
y_train= np.asarray(y_train)
X_test=np.array(X_test)
y_test=np.array(y_test)

In [0]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((985, 180), (985,), (329, 180), (329,))

In [0]:
x_traincnn = np.expand_dims(X_train, axis=2)
x_testcnn = np.expand_dims(X_test, axis=2)

In [0]:
x_traincnn.shape,x_testcnn.shape

((985, 180, 1), (329, 180, 1))

In [0]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

model = Sequential()

model.add(Conv1D(128, 5,padding='same',input_shape=(180,1)))        #1
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))



model.add(Conv1D(128, 5,padding='same',))                           #2
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(8))                                                 #3
model.add(Activation('softmax'))
opt = keras.optimizers.rmsprop(lr=0.00005, rho=0.9, epsilon=None, decay=0.0)

In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 180, 128)          768       
_________________________________________________________________
activation_4 (Activation)    (None, 180, 128)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 180, 128)          0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 22, 128)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 22, 128)           82048     
_________________________________________________________________
activation_5 (Activation)    (None, 22, 128)           0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 22, 128)          

In [0]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [0]:
cnnhistory=model.fit(x_traincnn, y_train, batch_size=20, epochs=500, validation_data=(x_testcnn, y_test))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 985 samples, validate on 329 samples
Epoch 1/500





985/985 [==============================] - 14s 14ms/step - loss: 3.0927 - acc: 0.2528 - val_loss: 1.4706 - val_acc: 0.4012
Epoch 2/500
985/985 [==============================] - 0s 196us/step - loss: 2.7424 - acc: 0.2883 - val_loss: 1.3045 - val_acc: 0.3830
Epoch 3/500
985/985 [==============================] - 0s 203us/step - loss: 2.3785 - acc: 0.3381 - val_loss: 1.3437 - val_acc: 0.4073
Epoch 4/500
985/985 [==============================] - 0s 227us/step - loss: 2.4133 - acc: 0.3279 - val_loss: 1.3388 - val_acc: 0.4650
Epoch 5/500
985/985 [==============================] - 0s 195us/step - loss: 2.1721 - acc: 0.3472 - val_loss: 1.3708 - val_acc: 0.3191
Epoch 6/500
985/985 [==============================] - 0s 211us/step - loss: 2.0986 - acc: 0.3482 - val_loss: 1.2896 - val_acc: 0.4711
Epoch 7/500
985/985 [=====================

In [0]:
em=['happy','sad','neutral','angry']

In [0]:
predictions = model.predict_classes(x_testcnn)
n=predictions[1]
print(em[n])
print(n)

angry
3


In [0]:
loss, acc = model.evaluate(x_testcnn, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

329/329 [==============================] - 0s 356us/step
Restored model, accuracy: 82.98%


In [0]:
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 
predictions = model.predict_classes(x_testcnn)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

         0.0       0.75      0.81      0.78        94
         1.0       0.88      0.83      0.86       101
         3.0       0.82      0.73      0.77        44
         4.0       0.87      0.90      0.89        90

    accuracy                           0.83       329
   macro avg       0.83      0.82      0.82       329
weighted avg       0.83      0.83      0.83       329

[[76  4  4 10]
 [12 84  3  2]
 [ 6  6 32  0]
 [ 8  1  0 81]]


In [0]:
filename = "/content/drive/My Drive/wav/Actor_02/03-01-01-01-02-01-02.wav"
    # record the file (start talking)
    #record_to_file(filename)
    # extract features and reshape it
features = np.array(extract_feature(filename, mfcc=True, chroma=True, mel=True).reshape(1, -1))
    # predict
f=np.expand_dims(features,axis=2)
result = model.predict_classes(f)[0]
    # show the result !
print("result :",em[result])

result : angry


**Second**

In [0]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

um = Sequential()

um.add(Conv1D(128, 5,padding='same',input_shape=(180,1)))#1
um.add(Activation('relu'))
um.add(Dropout(0.25))
um.add(MaxPooling1D(pool_size=(8)))

um.add(Conv1D(128, 5,padding='same',))                  #2
um.add(Activation('relu'))
um.add(MaxPooling1D(pool_size=(8)))
um.add(Dropout(0.25))

um.add(Conv1D(128, 5,padding='same',))                  #3
um.add(Activation('relu'))
um.add(Dropout(0.25))

um.add(Flatten())
um.add(Dense(8))                                        #4                      
um.add(Activation('softmax'))
opt = keras.optimizers.rmsprop(lr=0.00005,epsilon=None,rho=0.9,decay=0.0)



In [0]:
um.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 180, 128)          768       
_________________________________________________________________
activation_7 (Activation)    (None, 180, 128)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 180, 128)          0         
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 22, 128)           0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 22, 128)           82048     
_________________________________________________________________
activation_8 (Activation)    (None, 22, 128)           0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 2, 128)           

In [0]:
um.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [0]:
umhistory=um.fit(x_traincnn, y_train, batch_size=20, epochs=500, validation_data=(x_testcnn, y_test))

Train on 985 samples, validate on 329 samples
Epoch 1/500
985/985 [==============================] - 1s 599us/step - loss: 8.6718 - acc: 0.2406 - val_loss: 6.0213 - val_acc: 0.2888
Epoch 2/500
985/985 [==============================] - 0s 299us/step - loss: 8.0927 - acc: 0.2599 - val_loss: 6.0650 - val_acc: 0.2918
Epoch 3/500
985/985 [==============================] - 0s 233us/step - loss: 7.4811 - acc: 0.2640 - val_loss: 3.8898 - val_acc: 0.3161
Epoch 4/500
985/985 [==============================] - 0s 232us/step - loss: 5.5528 - acc: 0.2731 - val_loss: 1.5094 - val_acc: 0.3283
Epoch 5/500
985/985 [==============================] - 0s 255us/step - loss: 4.5710 - acc: 0.2883 - val_loss: 1.4401 - val_acc: 0.3435
Epoch 6/500
985/985 [==============================] - 0s 249us/step - loss: 4.1435 - acc: 0.2437 - val_loss: 1.3358 - val_acc: 0.3799
Epoch 7/500
985/985 [==============================] - 0s 269us/step - loss: 3.5226 - acc: 0.2751 - val_loss: 1.2863 - val_acc: 0.3769
Epoch 8/5

In [0]:
loss, acc = um.evaluate(x_testcnn, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

329/329 [==============================] - 0s 115us/step
Restored model, accuracy: 75.68%


In [0]:
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 
prediction = um.predict_classes(x_testcnn)
print(classification_report(y_test,prediction))
print(confusion_matrix(y_test,prediction))

              precision    recall  f1-score   support

         0.0       0.77      0.66      0.71        94
         1.0       0.81      0.70      0.75       101
         3.0       0.67      0.77      0.72        44
         4.0       0.75      0.91      0.82        90

    accuracy                           0.76       329
   macro avg       0.75      0.76      0.75       329
weighted avg       0.76      0.76      0.75       329

[[62  9  5 18]
 [ 9 71 12  9]
 [ 4  6 34  0]
 [ 6  2  0 82]]


**Third**

In [0]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

tm = Sequential()

tm.add(Conv1D(128, 5,padding='same',input_shape=(180,1)))#1
tm.add(Activation('relu'))
tm.add(Dropout(0.1))
tm.add(MaxPooling1D(pool_size=(8)))

tm.add(Conv1D(128, 5,padding='same',))                  #2
tm.add(Activation('relu'))
tm.add(MaxPooling1D(pool_size=(8)))
tm.add(Dropout(0.1))

tm.add(Conv1D(128, 5,padding='same',))                  #3
tm.add(Activation('relu'))
tm.add(Dropout(0.1))

tm.add(Flatten())
tm.add(Dense(8))                                        #4                      
tm.add(Activation('softmax'))
opt = keras.optimizers.rmsprop(lr=0.00005,epsilon=None,rho=0.9,decay=0.0)



In [0]:
tm.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_8 (Conv1D)            (None, 180, 128)          768       
_________________________________________________________________
activation_11 (Activation)   (None, 180, 128)          0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 180, 128)          0         
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 22, 128)           0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 22, 128)           82048     
_________________________________________________________________
activation_12 (Activation)   (None, 22, 128)           0         
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 2, 128)           

In [0]:
tm.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [0]:
tmhistory=tm.fit(x_traincnn, y_train, batch_size=20, epochs=500, validation_data=(x_testcnn, y_test))

Train on 985 samples, validate on 329 samples
Epoch 1/500
985/985 [==============================] - 1s 688us/step - loss: 4.6787 - acc: 0.2558 - val_loss: 1.9738 - val_acc: 0.3252
Epoch 2/500
985/985 [==============================] - 0s 266us/step - loss: 3.3170 - acc: 0.2863 - val_loss: 1.4963 - val_acc: 0.4195
Epoch 3/500
985/985 [==============================] - 0s 244us/step - loss: 2.9201 - acc: 0.2832 - val_loss: 1.5774 - val_acc: 0.2827
Epoch 4/500
985/985 [==============================] - 0s 252us/step - loss: 2.4707 - acc: 0.3310 - val_loss: 1.2878 - val_acc: 0.4073
Epoch 5/500
985/985 [==============================] - 0s 262us/step - loss: 2.1388 - acc: 0.3462 - val_loss: 1.3150 - val_acc: 0.3799
Epoch 6/500
985/985 [==============================] - 0s 272us/step - loss: 2.0049 - acc: 0.3310 - val_loss: 1.4397 - val_acc: 0.3313
Epoch 7/500
985/985 [==============================] - 0s 247us/step - loss: 1.8435 - acc: 0.3442 - val_loss: 1.1895 - val_acc: 0.4438
Epoch 8/5

In [0]:
loss, acc = tm.evaluate(x_testcnn, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

329/329 [==============================] - 0s 79us/step
Restored model, accuracy: 80.24%


In [0]:
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 
predict = tm.predict_classes(x_testcnn)
print(classification_report(y_test,predict))
print(confusion_matrix(y_test,predict))

              precision    recall  f1-score   support

         0.0       0.81      0.78      0.79        94
         1.0       0.81      0.77      0.79       101
         3.0       0.63      0.70      0.67        44
         4.0       0.87      0.91      0.89        90

    accuracy                           0.80       329
   macro avg       0.78      0.79      0.79       329
weighted avg       0.80      0.80      0.80       329

[[73  7  5  9]
 [ 7 78 13  3]
 [ 4  9 31  0]
 [ 6  2  0 82]]


**Fourth**

In [0]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

sm = Sequential()

sm.add(Conv1D(128, 5,padding='same',input_shape=(180,1)))#1
sm.add(Activation('relu'))
sm.add(Dropout(0.1))
sm.add(MaxPooling1D(pool_size=(8)))

sm.add(Conv1D(128, 5,padding='same',))                  #2
sm.add(Activation('relu'))
sm.add(MaxPooling1D(pool_size=(8)))
sm.add(Dropout(0.1))

sm.add(Conv1D(128, 5,padding='same',))                  #3
sm.add(Activation('relu'))
sm.add(Dropout(0.1))

sm.add(Conv1D(128, 5,padding='same',))                  #4
sm.add(Activation('relu'))
sm.add(Dropout(0.1))

sm.add(Flatten())
sm.add(Dense(8))                                        #5                     
sm.add(Activation('softmax'))
opt = keras.optimizers.rmsprop(lr=0.00005,epsilon=None,rho=0.9,decay=0.0)


In [0]:
sm.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_11 (Conv1D)           (None, 180, 128)          768       
_________________________________________________________________
activation_15 (Activation)   (None, 180, 128)          0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 180, 128)          0         
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 22, 128)           0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 22, 128)           82048     
_________________________________________________________________
activation_16 (Activation)   (None, 22, 128)           0         
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 2, 128)           

In [0]:
sm.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [0]:
smhistory=sm.fit(x_traincnn, y_train, batch_size=20, epochs=500, validation_data=(x_testcnn, y_test))

Train on 985 samples, validate on 329 samples
Epoch 1/500
985/985 [==============================] - 1s 839us/step - loss: 2.5278 - acc: 0.2650 - val_loss: 1.3942 - val_acc: 0.4286
Epoch 2/500
985/985 [==============================] - 0s 279us/step - loss: 2.0187 - acc: 0.3046 - val_loss: 1.3052 - val_acc: 0.3708
Epoch 3/500
985/985 [==============================] - 0s 284us/step - loss: 1.8336 - acc: 0.3015 - val_loss: 1.4177 - val_acc: 0.3495
Epoch 4/500
985/985 [==============================] - 0s 296us/step - loss: 1.6250 - acc: 0.3289 - val_loss: 1.2904 - val_acc: 0.3769
Epoch 5/500
985/985 [==============================] - 0s 283us/step - loss: 1.5385 - acc: 0.3635 - val_loss: 1.3058 - val_acc: 0.3982
Epoch 6/500
985/985 [==============================] - 0s 283us/step - loss: 1.4862 - acc: 0.3563 - val_loss: 1.3007 - val_acc: 0.3830
Epoch 7/500
985/985 [==============================] - 0s 291us/step - loss: 1.4838 - acc: 0.3218 - val_loss: 1.2472 - val_acc: 0.3891
Epoch 8/5

In [0]:
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 
p = sm.predict_classes(x_testcnn)
print(classification_report(y_test,p))
print(confusion_matrix(y_test,p))

              precision    recall  f1-score   support

         0.0       0.75      0.81      0.78        94
         1.0       0.80      0.89      0.85       101
         3.0       0.78      0.64      0.70        44
         4.0       0.94      0.83      0.88        90

    accuracy                           0.82       329
   macro avg       0.82      0.79      0.80       329
weighted avg       0.82      0.82      0.82       329

[[76 10  3  5]
 [ 6 90  5  0]
 [ 5 11 28  0]
 [14  1  0 75]]


In [0]:
loss, acc = sm.evaluate(x_testcnn, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

329/329 [==============================] - 0s 130us/step
Restored model, accuracy: 81.76%
